In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, get_scheduler
from bitsandbytes.optim import Adam8bit,PagedAdam32bit
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig
from peft import prepare_model_for_kbit_training
import torch
from IPython.display import  clear_output
import time
import gc,os
from torch.utils.data import Dataset, DataLoader
import numpy as np
from evaluation_utlis import rouge_bleu_score

device = 'cuda' if torch.cuda.is_available() else 'cpu'

DEFAULT_MODEL = "meta-llama/Llama-3.2-3B-Instruct"
TOKENIZER_PATH = DEFAULT_MODEL#"llama_odia_tokenizer"

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=False,
    )


model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    quantization_config=bnb_config,
    use_safetensors=True,
    device_map=device,
)

print(model.get_memory_footprint()/(1024*1024)) 

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH, use_safetensors=True)
tokenizer.pad_token_id = tokenizer.eos_token_id

def flush():
    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2095.841064453125


In [4]:
from datasets import load_dataset

# Load the dataset
# dataset = load_dataset("OdiaGenAI/hardcode_odia_qa_105")
dataset = load_dataset('OdiaGenAI/odia_domain_context_train_v1')

In [5]:
res = rouge_bleu_score(model,tokenizer,dataset['train'],device='cuda')

/home/mohan.dash/miniconda3/envs/diffusion_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/mohan.dash/miniconda3/envs/diffusion_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`

In [23]:
import evaluate
rouge = evaluate.load("rouge")

In [24]:
dataset['train'][0]

{'instruction': 'ଓଡ଼ିଶାର ଅର୍ଥନୀତି ଉପରେ ପର୍ଯ୍ୟଟନର କିଭଳି ପ୍ରଭାବ ପଡ଼ିଛି?',
 'input': '',
 'output': 'ବିଗତ କିଛି ବର୍ଷ ମଧ୍ୟରେ ପର୍ଯ୍ୟଟନ ଶିଳ୍ପ ଦ୍ୱାରା ଓଡ଼ିଶାର ଅର୍ଥନୀତି ବ୍ୟାପକ ଭାବେ ପ୍ରଭାବିତ ହୋଇଛି। ରାଜ୍ୟରେ ପ୍ରାକୃତିକ ସୌନ୍ଦର୍ଯ୍ୟ ଏବଂ ସାଂସ୍କୃତିକ ଐତିହ୍ୟ ସ୍ଥାନ ରହିଛି, ଯାହା ଦେଶ ଏବଂ ଦେଶ ବାହାରର ପର୍ଯ୍ୟଟକଙ୍କ ପାଇଁ ଆକର୍ଷଣୀୟ ହୋଇପାରିଛି।\nପର୍ଯ୍ୟଟନ କ୍ଷେତ୍ରରେ କାର୍ଯ୍ୟ କରୁଥିବା ଲୋକମାନଙ୍କ ପାଇଁ ନିଯୁକ୍ତି ସୁଯୋଗ ଏବଂ ଆୟ ବୃଦ୍ଧି ପାଇଛି, ଯେଉଁଥିରେ ଆତିଥ୍ୟ, ଟୁର ଅପରେଟର ଏବଂ ପରିବହନ ଶିଳ୍ପ ସାମିଲ ରହିଛି। ପର୍ଯ୍ୟଟନ ଶିଳ୍ପ ସ୍ଥାନୀୟ ଖାଦ୍ୟ, ହସ୍ତଶିଳ୍ପ ଏବଂ ସାଂସ୍କୃତିକ ଗତିବିଧିକୁ ପ୍ରୋତ୍ସାହିତ କରିବାରେ ସହାୟତା କରିଛି, ଯାହାକି ଓଡ଼ିଶାର ସାମଗ୍ରିକ ଅର୍ଥନୀତି ଉପରେ ସକାରାତ୍ମକ ପ୍ରଭାବ ପକାଇଛି।\nପର୍ଯ୍ୟଟକମାନଙ୍କ ପାଇଁ ଉତ୍ତମ ଅନୁଭୂତି ସୁନିଶ୍ଚିତ କରିବା ପାଇଁ ଭିତ୍ତିଭୂମି ବିକାଶ ଉପରେ ମଧ୍ୟ ସରକାର ଗୁରୁତ୍ୱ ଦେଉଛନ୍ତି। ସଡ଼କ ଏବଂ ସାର୍ବଜନୀନ ପରିବହନ ବ୍ୟବସ୍ଥାରେ ସୁଧାର କରାଯାଇଛି, ଯେତେବେଳେ କି ରାଜ୍ୟର ଅନେକ କ୍ଷେତ୍ରରେ ବିଭିନ୍ନ ଇକୋ-ଟୁରିଜମ ପ୍ରକଳ୍ପର ବିକାଶ କରାଯାଇଛି।\nଶେଷରେ ପର୍ଯ୍ୟଟନ ଓଡ଼ିଶାର ଆର୍ଥିକ ବିକାଶକୁ ତ୍ୱରାନ୍ୱିତ କରିବାରେ ଗୁରୁତ୍ୱପୂର୍ଣ୍ଣ ଭୂମିକା ନିର୍ବାହ କରିଛି ଏବଂ ରାଜ୍ୟବାସୀଙ୍କ ପାଇଁ ଅନେକ ଲାଭ ଆଣିପାରିଛି।'}

In [28]:
rouge.compute(predictions=['ଓଡ଼ିଶାର ଅର୍ଥନୀତି ଉପରେ ପର୍ଯ୍ୟଟନର କିଭଳି ପ୍ରଭାବ ପଡ଼ିଛି?'], 
              references=['ଓଡ଼ିଶାର '],
              tokenizer=lambda x: x.split(),  # Tokenize by whitespace only
              )

{'rouge1': 0.25, 'rouge2': 0.0, 'rougeL': 0.25, 'rougeLsum': 0.25}